In [2]:
from os import listdir
from scipy.io.wavfile import read
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.decomposition import PCA

# Preparing the data

In [3]:
def ls(path):
    array = listdir('musics_WAV/.')
    ans = []
    for i in array:
        if i.startswith('.') == False:
            ans.append(i)
    return ans

In [4]:
check_data = False
data = None

min_len = None
y = []

In [5]:
for i in ls('musics_WAV/.'):
    music = read('musics_WAV/' + str(i))
    music_in_numpy = np.array(music[1],dtype=float)

    if min_len == None:
        min_len = len(music_in_numpy)
    else:
        if len(music_in_numpy) < min_len:
            min_len = len(music_in_numpy)
print(min_len)

8423424


In [6]:
test_mus = []

In [7]:
for j in ls('musics_WAV/.'):
    music = read('musics_WAV/' + str(j))
    music_in_numpy = np.array(music[1],dtype=float)
    
    index = 0
    for i in range(min_len // 200000):
        for l in range(index, index + 200000, 50000):
            pca = PCA(n_components=1)
            if check_data == False:
                check_data = True
                m_part = music_in_numpy[l:l + 200000]
                m_part = pca.fit_transform(m_part)
                data = m_part
            else:
                m_part = music_in_numpy[l:l+200000]
                if len(data.T) == 222:
                    test_mus = music_in_numpy[l:l+200000]
                m_part = pca.fit_transform(m_part)
                data = np.concatenate((data, m_part), axis = 1)        
            y.append(int(j.replace('.wav', '')))
            if i == (min_len // 200000 - 1):
                break
                
            
        index += 200000
        

In [11]:
shuffler = np.random.permutation(len(data.T))

In [12]:
X = data.T[shuffler]
y = np.array(y)[shuffler]
X.shape

(825, 200000)

In [13]:
X_b = data.T
y_b = np.array(y)

In [18]:
from scipy.io.wavfile import write
test_mus.shape
scaled = np.int16(test_mus[0:(200000)]/np.max(np.abs(test_mus[0:(200000)])) * 32767)
string = str(0) + '.wav'
write(string, 44100, scaled)

# Create and fit model

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [65]:
model = keras.Sequential()
model.add(Dense(X.shape[0], activation='relu'))
model.add(Dense((X.shape[0] + len(ls('musics_WAV/.'))) // 2, activation='relu'))
model.add(Dense(len(ls('musics_WAV/.')), activation='sigmoid'))

In [66]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [67]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
21/21 [==============================] - 13s 541ms/step - loss: 162212.5077 - accuracy: 0.1649
Epoch 2/200
21/21 [==============================] - 11s 532ms/step - loss: 100368.6932 - accuracy: 0.7379
Epoch 3/200
21/21 [==============================] - 11s 530ms/step - loss: 15335.2405 - accuracy: 0.9009
Epoch 4/200
21/21 [==============================] - 11s 534ms/step - loss: 1128.8348 - accuracy: 0.9725
Epoch 5/200
21/21 [==============================] - 11s 535ms/step - loss: 687.7807 - accuracy: 0.9862
Epoch 6/200
21/21 [==============================] - 11s 528ms/step - loss: 14.8749 - accuracy: 0.9991
Epoch 7/200
21/21 [==============================] - 11s 536ms/step - loss: 230.8697 - accuracy: 0.9852
Epoch 8/200
21/21 [==============================] - 11s 536ms/step - loss: 339.9763 - accuracy: 0.9903
Epoch 9/200
21/21 [==============================] - 12s 568ms/step - loss: 249.0833 - accuracy: 0.9965
Epoch 10/200
21/21 [==============================] - 11

21/21 [==============================] - 11s 540ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 79/200
21/21 [==============================] - 11s 547ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 80/200
21/21 [==============================] - 12s 568ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 81/200
21/21 [==============================] - 12s 581ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 82/200
21/21 [==============================] - 12s 585ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 83/200
21/21 [==============================] - 13s 602ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 84/200
21/21 [==============================] - 12s 556ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 85/200
21/21 [==============================] - 12s 549ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 86/200
21/21 [==============================] - 11s 544ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 87/200
21/21 [==============================

In [68]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Accuracy = ' + str(test_acc * 100) + '%')

6/6 [==============================] - 0s 58ms/step - loss: 131179.1562 - accuracy: 0.2303
Accuracy = 23.030303418636322%


In [26]:
model.save('project_model')

INFO:tensorflow:Assets written to: project_model/assets


In [70]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_pred

array([3, 1, 0, 4, 0, 0, 2, 1, 4, 1, 3, 0, 3, 2, 1, 1, 2, 1, 1, 3, 1, 1,
       2, 1, 0, 0, 0, 1, 2, 4, 1, 0, 4, 1, 2, 3, 4, 3, 3, 1, 3, 0, 3, 1,
       0, 1, 0, 1, 3, 2, 2, 1, 1, 0, 1, 2, 0, 1, 0, 4, 0, 3, 4, 2, 1, 1,
       1, 0, 0, 1, 3, 4, 1, 1, 1, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 0, 4, 1,
       0, 4, 0, 0, 3, 0, 1, 1, 0, 1, 3, 1, 3, 1, 2, 0, 1, 4, 1, 4, 1, 0,
       3, 4, 0, 2, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 0, 0, 1, 0, 1, 0, 4,
       0, 3, 1, 4, 3, 0, 1, 4, 0, 3, 2, 0, 1, 2, 2, 0, 0, 1, 1, 3, 0, 0,
       1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1])

In [77]:
X_test[22]
y_test[22]

2

In [71]:
np.argmax(model.predict(X_test), axis=-1)

array([3, 1, 0, 4, 0, 0, 2, 1, 4, 1, 3, 0, 3, 2, 1, 1, 2, 1, 1, 3, 1, 1,
       2, 1, 0, 0, 0, 1, 2, 4, 1, 0, 4, 1, 2, 3, 4, 3, 3, 1, 3, 0, 3, 1,
       0, 1, 0, 1, 3, 2, 2, 1, 1, 0, 1, 2, 0, 1, 0, 4, 0, 3, 4, 2, 1, 1,
       1, 0, 0, 1, 3, 4, 1, 1, 1, 0, 1, 2, 0, 2, 1, 1, 1, 1, 0, 0, 4, 1,
       0, 4, 0, 0, 3, 0, 1, 1, 0, 1, 3, 1, 3, 1, 2, 0, 1, 4, 1, 4, 1, 0,
       3, 4, 0, 2, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 0, 0, 1, 0, 1, 0, 4,
       0, 3, 1, 4, 3, 0, 1, 4, 0, 3, 2, 0, 1, 2, 2, 0, 0, 1, 1, 3, 0, 0,
       1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1])

In [72]:
y_test

array([1, 1, 1, 3, 0, 4, 4, 3, 4, 1, 2, 3, 2, 0, 1, 3, 1, 0, 0, 1, 2, 2,
       2, 0, 3, 0, 4, 2, 2, 4, 4, 4, 1, 1, 3, 2, 2, 3, 3, 4, 1, 2, 2, 0,
       4, 3, 2, 0, 4, 3, 1, 3, 2, 4, 1, 2, 1, 3, 1, 2, 4, 4, 0, 1, 4, 1,
       1, 2, 1, 4, 4, 4, 0, 0, 1, 1, 3, 1, 4, 1, 2, 2, 1, 3, 1, 2, 2, 4,
       0, 4, 3, 0, 3, 0, 1, 1, 1, 4, 1, 0, 3, 0, 1, 4, 2, 3, 3, 4, 2, 1,
       3, 3, 3, 4, 0, 2, 0, 0, 3, 3, 1, 0, 1, 4, 2, 1, 0, 1, 0, 4, 4, 2,
       4, 1, 1, 2, 4, 0, 4, 2, 0, 2, 0, 3, 2, 4, 3, 2, 4, 4, 2, 3, 0, 3,
       2, 0, 2, 2, 0, 0, 1, 2, 4, 0, 1])

In [76]:
for i in range(len(y_test)):
    if y_test[i] == y_pred[i]:
        print(i, y_test[i])

1 1
4 0
8 4
9 1
14 1
22 2
25 0
28 2
29 4
33 1
37 3
38 3
54 1
55 2
65 1
66 1
71 4
74 1
82 1
88 0
89 4
91 0
92 3
93 0
94 1
95 1
100 3
107 4
110 3
118 3
126 0
127 1
128 0
134 1
137 0
140 0
151 3
152 0
160 1
163 0
164 1


In [88]:
X[22]

array([-2792.06187373, -2663.45681036, -2400.10629541, ...,
        -376.3572956 ,  -321.75802445,  -641.60727461])

In [110]:
test_mus

array([ 6.96924054e+00, -1.31537474e+02,  1.21136125e+03, -6.82202869e+02,
       -2.01718917e+02, -4.58251443e+02,  7.28194556e+02,  1.03875219e+03,
        2.88443974e+03,  1.28800319e+03,  6.26166369e+03, -1.63332434e+04,
        7.76131363e+03,  1.08519372e+04, -4.34246874e+03,  4.06878905e+03,
       -8.26640864e+02, -3.90605246e+02, -4.33630245e+03, -5.58428107e+03,
        1.91739053e+02,  6.36049309e+03, -2.31870621e+04, -2.90905605e+04,
       -3.91364358e+03,  3.61797440e+03, -2.20222605e+04, -7.94559145e+02,
       -4.90494203e+03,  3.11232189e+03,  2.67296120e+03, -6.78404947e+03,
       -1.23250712e+04, -1.21229040e+04, -1.51399586e+03, -1.30760943e+04,
        1.54751547e+04,  1.44844018e+04, -3.60108698e+02, -1.66538387e+03,
       -9.42167770e+03, -1.52872846e+04, -5.05767601e+03,  1.60860548e+04,
        1.00955482e+04,  6.20234096e+03, -1.77325026e+04,  2.85525470e+03,
       -1.57984873e+04, -1.47309064e+04, -6.82014096e+03, -4.22589865e+03,
        1.70596332e+03,  